In [1]:
# %%
from pathlib import Path
import pandas as pd

pd.set_option("mode.copy_on_write", True)

repo_root = Path("../..")
# source files
src_folder = repo_root / "00_source_data/population_data"
src09 = src_folder / "co-est00int-tot.csv"
src19 = src_folder / "co-est2020.csv"

# read into pandas dataframe
# 09 and 19 as we only need 00-09 from the first, 10-19 from the second
pop09 = pd.read_csv(src09, encoding="latin-1")
pop19 = pd.read_csv(src19, encoding="latin-1")

# output file
out_file = repo_root / "20_intermediate_file/population_00-19.csv"

In [2]:
pop09.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME',
       'ESTIMATESBASE2000', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP',
       'POPESTIMATE2010'],
      dtype='object')

In [3]:
def clean_pop(pop):
    # clean rows: drop state and Alaska rows
    pop = pop[(pop["SUMLEV"] == 50) & (pop["STNAME"] != "Alaska")]
    pop = pop.reset_index(drop=True)

    # create FIPS column (int values)
    pop["FIPS"] = pop["STATE"] * 1000 + pop["COUNTY"]
    return pop

In [4]:
# clean pop09
pop09 = clean_pop(pop09)

# drop columns no longer needed
pop09.drop(
    inplace=True,
    columns=[
        "SUMLEV",
        "REGION",
        "DIVISION",
        "STATE",
        "COUNTY",
        "ESTIMATESBASE2000",
        "CENSUS2010POP",
        "POPESTIMATE2010",
    ],
)
pop09.rename(inplace=True, columns=lambda c: c[-4:] if "POP" in c else c)

In [5]:
# clean pop19
pop19 = clean_pop(pop19)

# drop columns no longer needed
pop19.drop(
    inplace=True,
    columns=[
        "SUMLEV",
        "REGION",
        "DIVISION",
        "STATE",
        "COUNTY",
        "ESTIMATESBASE2010",
        "CENSUS2010POP",
        "POPESTIMATE042020",
        "POPESTIMATE2020",
    ],
)
pop19.rename(inplace=True, columns=lambda c: c[-4:] if "POP" in c else c)

In [6]:
set(pop09["FIPS"]) - set(pop19["FIPS"])

{46113, 51515}

In [7]:
set(pop19["FIPS"]) - set(pop09["FIPS"])

{46102}

In [8]:
pop09[pop09["FIPS"].isin([46113, 51515])]

,STNAME,CTYNAME,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,FIPS
2388,South Dakota,Shannon County,12542,12602,12872,12993,12983,13150,13404,13345,13368,13425,46113
2887,Virginia,Bedford city,6425,6400,6320,6237,6198,6186,6147,6169,6150,6174,51515


In [9]:
pop19[pop19["FIPS"] == 46102]

,STNAME,CTYNAME,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,FIPS
2383,South Dakota,Oglala Lakota County,13636,13897,14041,14130,14217,14364,14426,14392,14324,14171,46102


In [10]:
pop19.loc[pop19["FIPS"] == 46102, ["CTYNAME", "FIPS"]] = ["Shannon County", 46113]

In [11]:
pop19[pop19["FIPS"] == 46113]

,STNAME,CTYNAME,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,FIPS
2383,South Dakota,Shannon County,13636,13897,14041,14130,14217,14364,14426,14392,14324,14171,46113


In [12]:
id_vars = ["STNAME", "CTYNAME", "FIPS"]
pop09 = pop09.melt(id_vars=id_vars, var_name="Year", value_name="Population")
pop19 = pop19.melt(id_vars=id_vars, var_name="Year", value_name="Population")

In [13]:
pop_all = pd.concat([pop09, pop19])
pop_all

,STNAME,CTYNAME,FIPS,Year,Population
0,Alabama,Autauga County,1001,2000,44021
1,Alabama,Baldwin County,1003,2000,141342
2,Alabama,Barbour County,1005,2000,29015
3,Alabama,Bibb County,1007,2000,19913
4,Alabama,Blount County,1009,2000,51107
...,...,...,...,...,...
31125,Wyoming,Sweetwater County,56037,2019,42917
31126,Wyoming,Teton County,56039,2019,23385
31127,Wyoming,Uinta County,56041,2019,20196
31128,Wyoming,Washakie County,56043,2019,7824


In [14]:
pop_all.to_csv(out_file, index=False)